In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from csv import reader
import pandas as pd

In [2]:
#load tokenizer and model 
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model =T5ForConditionalGeneration.from_pretrained('/Users/cayadehaas/PycharmProjects/Communicative_Robots/triple2text.pt', 
                                                return_dict=True, config='/Users/cayadehaas/PycharmProjects/Communicative_Robots/t5-base-config.json')
def generate(text,model,tokenizer):
    model.eval()
    input_ids = tokenizer.encode("WebNLG:{} </s>".format(text), 
                               return_tensors="pt")  
    outputs = model.generate(input_ids)
    
    return tokenizer.decode(outputs[0])

In [6]:
#create an empty dataframe with generated sentences as column
df = pd.DataFrame(columns=['generated_sentences'])
#open triple file in read mode
with open('/Users/cayadehaas/PycharmProjects/Communicative_Robots/output_with_sentences.csv', 'r') as read_obj:
    # pass the file object to reader() to get the reader object
    csv_reader = reader(read_obj)
    # Iterate over each row in the csv using reader object
    for row in csv_reader:
        #remove the brackets from each row to make it the right format into the tokenizer
        row = ''.join(row[-2])
        model.eval()
        input_ids = tokenizer.encode(row, return_tensors="pt")  # Batch size 1
        outputs = model.generate(input_ids)
        sentence = tokenizer.decode(outputs[0])
        #add generated sentences to dataframe
        df = df.append({'generated_sentences': sentence}, ignore_index=True)

df

<pad> The Motor Sport Sport Vision is located in Fawkham and its HQ is Faw


In [12]:
#separate the target_text from the test data
test_df=pd.read_csv('/Users/cayadehaas/PycharmProjects/Communicative_Robots/output_with_sentences.csv', header=None)
target_text = test_df[2]

In [13]:
# #create df from target_text
target_df = pd.DataFrame(columns=['target_sentences'])
target_df['target_sentences'] = target_text
target_df

,target_sentences
0,MotorSport Vision is located in the city of Fa...
1,MotorSport Vision is located in the city of Fa...
2,MotorSport Vision is located in Fawkham.
3,Ciudad Ayala is a city with population density...
4,"The city of Ciudad Ayala, with a population of..."
...,...
2617,"Member of the entertainment industry, The GMA ..."
2618,GMA New Media is located in the GMA Network Ce...
2619,Ciudad Ayala is in the country of Mexico and i...
2620,Ciudad Ayala is a Mexican city located in the ...


In [64]:
#concatenate results and save as csv
result = pd.concat([df, target_df], axis=1, join='inner')
result.to_csv('generated_sentences_vs_target_sentences.csv', mode='w', header=False)
result